In [42]:
import pandas as pd
import numpy as np
from collections import Counter

import time
import os
from sqlalchemy import create_engine
import psycopg2 as pg
import io
import re
from datetime import datetime, timedelta

## 1) Import Data

In [43]:
# Create SQLAlchemy Connection
engine = create_engine('postgresql://postgres:@localhost:5433/DeepREI')
connection = engine.raw_connection()

# Instantiate Cursor
cursor = connection.cursor()

In [44]:
# Import Raw Data
base_data = pd.read_sql_query("SELECT * FROM public.realtortable", connection)

### Clean Data

In [45]:
# Remove Columns we dont want
base_data = base_data[['ListingType', 'description', 'bedroom', 'FullBathrooms', 'HalfBathrooms','BathroomDescription', 'InteriorFeatures', 'CoolingFeatures','HeatingFeatures', 'DiningRoomDescription', 'ExteriorandLotFeatures','WaterFeatures', 'WaterfrontDescription', 'PoolFeatures','PoolDescription', 'ManufacturedMode', 'GarageSpaces','ParkingFeatures', 'HomeFeaturesView', 'OtherEquipment','ElementarySchool', 'HighSchool', 'MiddleSchool','AssociationFeeAmenitie', 'Association', 'AssociationAmenities','AssociationFee', 'AssociationFeeFrequency','CalculatedTotalMonthlyAssociation', 'MaintenanceDescription','PetDescription', 'SourceListingStatus', 'County','Directions', 'TaxYear', 'Restrictions', 'SourcePropertyType', 'Area','SourceNeighborhood', 'PostalCode', 'PublicSurveySection','Subdivision', 'Zoning', 'SourceSystemName', 'TotalSquareFeetLiving','YearBuilt', 'ConstructionMaterials', 'DirectionFaces', 'PropertyAge','Roof', 'LevelsorStories', 'StructureType', 'HouseStyle','TotalAreaSqft', 'YearBuiltDetails', 'ArchitecturalStyle', 'Sewer','WaterSource']]

# Replace Null with Nan
base_data_sold = base_data[base_data['ListingType']=='buy']

# Change Various Values
base_data[['YearBuilt','PropertyAge']] = base_data[['YearBuilt','PropertyAge']].replace('New Construction', 0)

# Change Columns to Numeric
base_data['bedroom'] = pd.to_numeric(base_data['bedroom'], errors='coerce')
base_data['FullBathrooms'] = pd.to_numeric(base_data['FullBathrooms'], errors='coerce')
base_data['HalfBathrooms'] = pd.to_numeric(base_data['HalfBathrooms'], errors='coerce')
base_data['GarageSpaces'] = pd.to_numeric(base_data['GarageSpaces'], errors='coerce')
base_data['YearBuilt'] = pd.to_numeric(base_data['YearBuilt'], errors='coerce')
base_data['PropertyAge'] = pd.to_numeric(base_data['PropertyAge'], errors='coerce')
base_data['TotalSquareFeetLiving'] = pd.to_numeric(base_data['TotalSquareFeetLiving'], errors='coerce')


In [46]:
# Column Names and their respective feature types
num_columns = ['bedroom','FullBathrooms', 'HalfBathrooms','GarageSpaces','YearBuilt', 'PropertyAge','TotalSquareFeetLiving']

cat_columns = ['CoolingFeatures','HeatingFeatures','ManufacturedMode','ElementarySchool','HighSchool', 'MiddleSchool','AssociationFeeFrequency','PetDescription',
                'County','Restrictions','Area','PublicSurveySection','Zoning','SourceSystemName','DirectionFaces','Roof','StructureType','HouseStyle','ArchitecturalStyle',
                'Sewer','WaterSource', 'SourceNeighborhood']

verbose_columns = ['BathroomDescription', 'InteriorFeatures', 'CoolingFeatures','HeatingFeatures', 'DiningRoomDescription', 'ExteriorandLotFeatures','WaterFeatures',                           'WaterfrontDescription', 'PoolFeatures','PoolDescription', 'ParkingFeatures', 'HomeFeaturesView', 'OtherEquipment', 'AssociationAmenities']

In [50]:
base_data.to_csv('Realtor Sample Data.csv', index=False)

## Change

In [90]:
class word_count_cat():

    def __init__(self, data, threshold, most_common):
        self.data = data
        self.threshold = threshold
        self.most_common = most_common

        # Lower case string, Remove Punctuation, and Split on ' '
        self.data['Writing'] = self.data['Writing'].apply(lambda x: re.sub(r'[^\w\s]','',x.lower()).split(' '))

        # Count all instances of a word
        results = Counter()
        self.data['Writing'].apply(results.update)

        # Make list of words over threshold if most_common = True
        if self.most_common:
            word_feats = [(x) for (x,y) in results.most_common() if y > self.threshold]

        # Else Make list of words below threshold if most_common = False
        else:    
            word_feats = [(x) for (x,y) in results.most_common() if y < self.threshold]
 
        # For Each word in list, make column and assign Binary value
        for word in word_feats:

            # Return binary, whether iteration is in list of strings
            self.data[f'writing_cat_{word}'] = self.data['Writing'].apply(lambda x: 1 if word in x else 0)


In [91]:
word_count_cat(data=df, threshold=6, most_common=True).data

,Member,Writing,writing_cat_the,writing_cat_that,writing_cat_to,writing_cat_a
0,1,"[last, month, i, wrote, an, article, about, so...",1,1,1,1
1,2,"[it, turns, out, that, the, python, programmin...",1,1,1,1
2,3,"[when, going, over, a, lot, of, these, tools, ...",1,1,1,1
3,4,"[this, is, an, incredibly, powerful, module, t...",0,1,1,1
4,5,"[probably, the, coolest, thing, that, the, fun...",1,1,1,1
